# 4. LangChain解説

In [ ]:
import os

from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

## 4-1 LangChainの概要

In [ ]:
# 2023年1月、LangChainのバージョン0.1がリリースされました。
# LangChainはバージョン0.1から、破壊的変更がある場合は事前通知とマイナーバージョンアップで対応されることになりました。
# また同時期に、langchainという1つのパッケージから、langchain-core・langchain・langchain-openaiなどのパッケージに分割されました。
#
# LangChainのバージョン0.1リリースの少し前までのアップデートについては、以下のスライドにまとめているので参考にしてください。
#
# スライド「速習：LangChainの大きなアップデート（2023年秋〜冬）」
# https://speakerdeck.com/os1ma/su-xi-langchainnoda-kinaatupudeto-2023nian-qiu-dong

!pip install langchain-core==0.1.18 langchain==0.1.5 langchain-openai==0.0.5

## 4-2 Language models

### LLMs

In [ ]:
from langchain_openai import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0)

result = llm.invoke("自己紹介してください。")
print(result)

### Chat models

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="こんにちは！私はジョンと言います！"),
    AIMessage(content="こんにちは、ジョンさん！どのようにお手伝いできますか？"),
    HumanMessage(content= "私の名前が分かりますか？")
]

result = chat.invoke(messages)
print(result.content)

### Callbackを使ったストリーミング

In [ ]:
# ストリーミングはCallbackではなく、streamメソッドで実行可能になりました。

from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

messages = [HumanMessage(content="自己紹介してください")]

for chunk in chat.stream(messages):
    print(chunk.content)

## 4-3 Prompts

### Prompt templates

In [ ]:
# PromptTemplateやChatPromptTemplateは、以前より簡単に使用可能になりました。

from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
以下の料理のレシピを考えてください。

料理名: {dish}
"""
)

result = prompt.format(dish="カレー")
print(result)

### ChatPromptTemplate

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは{country}料理のプロフェッショナルです。"),
    ("human", "以下の料理のレシピを考えてください。\n\n料理名: {dish}")
])

messages = chat_prompt.format_messages(country="イギリス", dish="肉じゃが")

print(messages)

In [ ]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
result = chat.invoke(messages)
print(result.content)

## 4-4 Output Parsers


### PydanticOutputParser

In [ ]:
from pydantic import BaseModel, Field

class Recipe(BaseModel):
   ingredients: list[str] = Field(description="ingredients of the dish")
   steps: list[str] = Field(description="steps to make the dish")

In [ ]:
from langchain.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=Recipe)

In [ ]:
format_instructions = parser.get_format_instructions()

print(format_instructions)

In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    template="""料理のレシピを考えてください。

{format_instructions}

料理名: {dish}
""",
    partial_variables={"format_instructions": format_instructions},
)

In [ ]:
formatted_prompt = prompt.format(dish="カレー")

print(formatted_prompt)

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
messages = [HumanMessage(content=formatted_prompt)]
output = chat.invoke(messages)

print(output.content)

In [ ]:
recipe = parser.invoke(output.content)
print(recipe)

## 4-4 Chains

### LLMChain―PromptTemplate・Language model・OutputParserをつなぐ

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

class Recipe(BaseModel):
   ingredients: list[str] = Field(description="ingredients of the dish")
   steps: list[str] = Field(description="steps to make the dish")

output_parser = PydanticOutputParser(pydantic_object=Recipe)

prompt = PromptTemplate.from_template(
    template="""料理のレシピを考えてください。

{format_instructions}

料理名: {dish}
""",
    partial_variables={"format_instructions": format_instructions},
)

model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
# 2023年10月後半頃から、LangChainでは「LangChain Expression Language (LCEL)」を使う実装が標準的となりました。
# LCELではプロンプトやLLMを `|` で繋げて書き、処理の連鎖 (Chain) を実装します。
# 以後のサンプルコードでは、LCELを使用します。
#
# LCELの概要は、公式ドキュメントや以下の記事を参考にしてください。
#
# 記事「LangChain の新記法「LangChain Expression Language (LCEL)」入門」
# https://zenn.dev/os1ma/articles/acd3472c3a6755

chain = prompt | model | output_parser

recipe = chain.invoke({"dish": "カレー"})

print(type(recipe))
print(recipe)

### SimpleSequentialChain―ChainとChainをつなぐ

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

cot_prompt = PromptTemplate.from_template(
    """以下の質問に回答してください。

質問: {question}

ステップバイステップで考えましょう。
"""
)

cot_chain = (
    {"question": RunnablePassthrough()}
    | cot_prompt
    | model
    | StrOutputParser()
)

In [ ]:
summarize_prompt = PromptTemplate.from_template(
    """以下の文章を結論だけ一言に要約してください。

{input}
"""
)

summarize_chain = (
    {"input": RunnablePassthrough()}
    | summarize_prompt
    | model
    | StrOutputParser()
)

In [ ]:
cot_summarize_chain = cot_chain | summarize_chain

result = cot_summarize_chain.invoke(
   "私は市場に行って10個のリンゴを買いました。隣人に2つ、修理工に2つ渡しました。それから5つのリンゴを買って1つ食べました。残りは何個ですか？"
)
print(result)

上記の入力は [Chain-of-Thoughtプロンプティング | Prompt Engineering Guide](https://www.promptingguide.ai/jp/techniques/cot) から引用しました。

## 4-5 Memory

### ConversationBufferMemory

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-4", temperature=0)
conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory()
)

while True:
    user_message = input("You: ")
    ai_message = conversation.predict(input=user_message)
    print(f"AI: {ai_message}")

In [ ]:
# ChatOpenAIとConversationBufferMemoryに加えて、
# 独自のプロンプトテンプレートを使用する例は次のようになります。

from langchain.chains import LLMChain
from langchain_core.prompts import MessagesPlaceholder

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

chain = LLMChain(llm=chat, prompt=prompt, memory=memory)

while True:
    user_message = input("You: ")
    result = chain.invoke({"input": user_message})
    ai_message = result["text"]
    print(ai_message)